In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses
import os
from tensorflow.keras import preprocessing

In [2]:
base_dir = '/Users/phoelapyae/Downloads/stack_overflow_16k'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [3]:
seed = 42
batch_size = 32

In [4]:
raw_train_ds = preprocessing.text_dataset_from_directory(train_dir, seed=42, validation_split=0.2, subset='training', batch_size=batch_size)
raw_val_ds = preprocessing.text_dataset_from_directory(train_dir, seed=42, validation_split=0.2, subset='validation', batch_size=batch_size)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.
Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [5]:
raw_test_ds = preprocessing.text_dataset_from_directory(test_dir, batch_size=batch_size)

Found 8000 files belonging to 4 classes.


In [6]:
import re, string

def custom_standardize(text):
    text = tf.strings.lower(text)
    stripped_text = tf.strings.regex_replace(text, '<br >', ' ')
    return tf.strings.regex_replace(stripped_text, '[%s]' % re.escape(string.punctuation), '')

In [7]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

max_features = 10000
maxlen = 300

vectorize_layer = TextVectorization(max_tokens=max_features, output_sequence_length=maxlen, output_mode='int', standardize=custom_standardize)

In [8]:
len(vectorize_layer.get_vocabulary())

0

In [9]:
train_data = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_data)

In [10]:
len(vectorize_layer.get_vocabulary())

10000

In [11]:
AUTOTUNE = tf.data.AUTOTUNE

raw_train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
raw_val_ds = raw_val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [12]:
model = models.Sequential([
    vectorize_layer,
    layers.Embedding(max_features + 1, 16),
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dense(16, activation='relu'),
    layers.Dense(4, activation='softmax')
])
model.compile(optimizer=optimizers.Adam(), loss=losses.SparseCategoricalCrossentropy(), metrics=['acc'])

In [13]:
history = model.fit(raw_train_ds, epochs=20, validation_data=raw_val_ds)

Epoch 1/20
200/200 [==============================] - 4s 16ms/step - loss: 1.3836 - acc: 0.2870 - val_loss: 1.3604 - val_acc: 0.3369
Epoch 2/20
200/200 [==============================] - 1s 7ms/step - loss: 1.3332 - acc: 0.4348 - val_loss: 1.2052 - val_acc: 0.5944
Epoch 3/20
200/200 [==============================] - 1s 7ms/step - loss: 1.1548 - acc: 0.5974 - val_loss: 1.0069 - val_acc: 0.6656
Epoch 4/20
200/200 [==============================] - 1s 7ms/step - loss: 0.9627 - acc: 0.6864 - val_loss: 0.8571 - val_acc: 0.7269
Epoch 5/20
200/200 [==============================] - 2s 8ms/step - loss: 0.8105 - acc: 0.7376 - val_loss: 0.7479 - val_acc: 0.7475
Epoch 6/20
200/200 [==============================] - 1s 7ms/step - loss: 0.6958 - acc: 0.7695 - val_loss: 0.6746 - val_acc: 0.7581
Epoch 7/20
200/200 [==============================] - 1s 7ms/step - loss: 0.6140 - acc: 0.7913 - val_loss: 0.6257 - val_acc: 0.7706
Epoch 8/20
200/200 [==============================] - 1s 7ms/step - loss: 0

In [14]:
model.evaluate(raw_test_ds)

250/250 [==============================] - 8s 31ms/step - loss: 0.6240 - acc: 0.7771


[0.6239853501319885, 0.7771250009536743]

In [25]:
texts, labels = next(iter(raw_test_ds))
first_question, first_label = texts[1], labels[1]
print(first_question)
print(first_label)

tf.Tensor(b'"hacking select multiple in blank, flicke i\'ve hacked html\'s &lt; select multiple > with blank, according to my customer\'s specifications:...clicking an item only toggles that item\'s selected status..other selected items stay selected....the little bit of blank remembers all selected values..when the user clicks, only the option he clicks will be selected..the blank selects the options he remembers...however, it causes a flicker effect. i doubt there is any solution for this, but i felt i had to ask just in case, does anyone know of a way to delay the rendering, or any other solution to accomplish this, without a flicker?..best regards...edit: here is the code..var choices=new array();.function prepmulti(){.    var m=document.queryselectorall(\'select\');.    for(var i=0;i&lt;m.length;i++).        if(m[i].id!=\'\'){.            m[i].onclick=toggle;.            choices.push(new array());.        }.}.function toggle(){.    var sel, x;.    for(var i=0; i&lt;this.options.le

In [26]:
import numpy as np

predict = model.predict(tf.expand_dims(first_question, -1))
print(np.argmax(predict))

2
